<a href="https://colab.research.google.com/github/krishnaaxo/AutoGluon_Automl/blob/main/AutoGluon_Automl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook purpose is to try out AutoGluon package open sourced by AWS. The project details is available in link below

https://autogluon.mxnet.io/


In [1]:
!pip install --upgrade mxnet
!pip install autogluon
!pip install --upgrade pip
!pip install --upgrade setuptools
!pip install --upgrade "mxnet<2.0.0"
!pip install --pre autogluon

In [2]:
import autogluon as ag
import pandas as pd
import numpy as np
import os,urllib

In [4]:

from autogluon.tabular import TabularPredictor as task

In [25]:
churn_master_df = pd.read_csv('/content/healthcare-dataset-stroke-data.csv')

In [26]:
size = int(0.8*churn_master_df.shape[0])
train_df = churn_master_df[:size]
test_df = churn_master_df[size:]

In [27]:
test_df.shape

(1022, 12)

In [28]:
test_df

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
4088,27705,Female,82.0,0,1,Yes,Self-employed,Rural,88.60,32.5,Unknown,0
4089,36850,Male,36.0,0,0,Yes,Govt_job,Urban,57.59,32.8,Unknown,0
4090,52500,Female,42.0,0,0,Yes,Govt_job,Urban,59.43,25.4,never smoked,0
4091,43698,Female,27.0,0,0,No,Govt_job,Rural,65.43,27.2,Unknown,0
4092,49901,Male,55.0,0,0,Yes,Govt_job,Urban,154.03,31.6,smokes,0
...,...,...,...,...,...,...,...,...,...,...,...,...
5105,18234,Female,80.0,1,0,Yes,Private,Urban,83.75,NaN,never smoked,0
5106,44873,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.0,never smoked,0
5107,19723,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.6,never smoked,0
5108,37544,Male,51.0,0,0,Yes,Private,Rural,166.29,25.6,formerly smoked,0


In [29]:
train_data = task.Dataset(train_df)
test_data = task.Dataset(test_df)

In [30]:
print(train_data.head())

      id  gender   age  ...   bmi   smoking_status stroke
0   9046    Male  67.0  ...  36.6  formerly smoked      1
1  51676  Female  61.0  ...   NaN     never smoked      1
2  31112    Male  80.0  ...  32.5     never smoked      1
3  60182  Female  49.0  ...  34.4           smokes      1
4   1665  Female  79.0  ...  24.0     never smoked      1

[5 rows x 12 columns]


In [31]:
print(test_data.describe())

                 id          age  ...         bmi  stroke
count   1022.000000  1022.000000  ...  998.000000  1022.0
mean   37272.861057    41.202270  ...   28.754910     0.0
std    20903.098419    22.171808  ...    7.913709     0.0
min       67.000000     0.240000  ...   13.300000     0.0
25%    19531.250000    23.000000  ...   23.500000     0.0
50%    38041.500000    42.000000  ...   27.900000     0.0
75%    54870.500000    58.000000  ...   32.900000     0.0
max    72940.000000    82.000000  ...   92.000000     0.0

[8 rows x 7 columns]


In [33]:
label_column = 'stroke'
train_data[label_column].describe()

count    4088.000000
mean        0.060910
std         0.239194
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: stroke, dtype: float64

In [36]:
predictor = task(label=label_column, eval_metric='accuracy').fit(train_data)

No path specified. Models will be saved in: "AutogluonModels/ag-20210518_135811/"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20210518_135811/"
AutoGluon Version:  0.2.0
Train Data Rows:    4088
Train Data Columns: 11
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [1, 0]
	If 'binary' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
NumExpr defaulting to 2 threads.
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    12678.31 MB
	Train Data (Original)  Memory Usage: 1.49 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to 

In [37]:
y_test = test_data[label_column]

test_data_nolab = test_data.drop(labels=[label_column],axis=1) 
print(test_data_nolab.head())


         id  gender   age  ...  avg_glucose_level   bmi smoking_status
4088  27705  Female  82.0  ...              88.60  32.5        Unknown
4089  36850    Male  36.0  ...              57.59  32.8        Unknown
4090  52500  Female  42.0  ...              59.43  25.4   never smoked
4091  43698  Female  27.0  ...              65.43  27.2        Unknown
4092  49901    Male  55.0  ...             154.03  31.6         smokes

[5 rows x 11 columns]


In [38]:
y_pred = predictor.predict(test_data_nolab)
print("Predictions:  ", y_pred)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Evaluation: accuracy on test data: 0.9951076320939335
Evaluations on test data:
{
    "accuracy": 0.9951076320939335,
    "balanced_accuracy": 0.9975538160469668,
    "mcc": 0.0,
    "f1": 0.0,
    "precision": 0.0,
    "recall": 0.0
}


Predictions:   4088    0
4089    0
4090    0
4091    0
4092    0
       ..
5105    1
5106    0
5107    0
5108    0
5109    0
Name: stroke, Length: 1022, dtype: int64


In [41]:
leaderboard = predictor.leaderboard(test_data)

                  model  score_test  score_val  pred_time_test  pred_time_val   fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0              CatBoost    1.000000      0.940        0.007990       0.008739   0.881503                 0.007990                0.008739           0.881503            1       True          7
1            LightGBMXT    1.000000      0.940        0.013570       0.015829   3.077721                 0.013570                0.015829           3.077721            1       True          3
2       NeuralNetFastAI    1.000000      0.940        0.055843       0.041229  18.399883                 0.055843                0.041229          18.399883            1       True         10
3        NeuralNetMXNet    0.999022      0.940        0.220773       0.178294  12.754510                 0.220773                0.178294          12.754510            1       True         12
4      RandomForestEntr    0.999022     

In [42]:
print(predictor.problem_type)
y_pred = predictor.predict(test_data)
print("Predictions:  ", list(y_pred)[:5])
perf = predictor.evaluate(test_data, auxiliary_metrics=False)

Evaluation: accuracy on test data: 0.9951076320939335
Evaluations on test data:
{
    "accuracy": 0.9951076320939335
}


binary
Predictions:   [0, 0, 0, 0, 0]


In [43]:
results = predictor.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                  model  score_val  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0               XGBoost      0.946       0.012189   0.702173                0.012189           0.702173            1       True         11
1   WeightedEnsemble_L2      0.946       0.014268   1.504321                0.002079           0.802147            2       True         14
2              LightGBM      0.944       0.016811   0.426232                0.016811           0.426232            1       True          4
3         LightGBMLarge      0.944       0.017528   0.934855                0.017528           0.934855            1       True         13
4      RandomForestGini      0.942       0.210637   1.373432                0.210637           1.373432            1       True          5
5              CatBoost      0.940       0.008739   0.881503                0.008739           0.881503 

In [44]:
predictor.predict_proba(test_data_nolab)

,0,1
4088,0.650719,0.349281
4089,0.698421,0.301579
4090,0.698421,0.301579
4091,0.698421,0.301579
4092,0.673214,0.326786
...,...,...
5105,0.430271,0.569729
5106,0.574649,0.425351
5107,0.698421,0.301579
5108,0.649492,0.350508


In [45]:
predictor.leaderboard()

                  model  score_val  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0               XGBoost      0.946       0.012189   0.702173                0.012189           0.702173            1       True         11
1   WeightedEnsemble_L2      0.946       0.014268   1.504321                0.002079           0.802147            2       True         14
2              LightGBM      0.944       0.016811   0.426232                0.016811           0.426232            1       True          4
3         LightGBMLarge      0.944       0.017528   0.934855                0.017528           0.934855            1       True         13
4      RandomForestGini      0.942       0.210637   1.373432                0.210637           1.373432            1       True          5
5              CatBoost      0.940       0.008739   0.881503                0.008739           0.881503            1       True          7
6            LightGBMXT    

,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,XGBoost,0.946,0.012189,0.702173,0.012189,0.702173,1,True,11
1,WeightedEnsemble_L2,0.946,0.014268,1.504321,0.002079,0.802147,2,True,14
2,LightGBM,0.944,0.016811,0.426232,0.016811,0.426232,1,True,4
3,LightGBMLarge,0.944,0.017528,0.934855,0.017528,0.934855,1,True,13
4,RandomForestGini,0.942,0.210637,1.373432,0.210637,1.373432,1,True,5
5,CatBoost,0.940,0.008739,0.881503,0.008739,0.881503,1,True,7
6,LightGBMXT,0.940,0.015829,3.077721,0.015829,3.077721,1,True,3
7,NeuralNetFastAI,0.940,0.041229,18.399883,0.041229,18.399883,1,True,10
8,KNeighborsUnif,0.940,0.105825,0.011422,0.105825,0.011422,1,True,1
9,NeuralNetMXNet,0.940,0.178294,12.754510,0.178294,12.754510,1,True,12


In [49]:
predictor = TabularPredictor(label=label_column, eval_metric='accuracy').fit(train_data,
    num_bag_folds=5, num_bag_sets=1, num_stack_levels=1,
    hyperparameters = {'NN': {'num_epochs': 2}, 'GBM': {'num_boost_round': 20}},  # last  argument is just for quick demo here, omit it in real applications
)

No path specified. Models will be saved in: "AutogluonModels/ag-20210518_140306/"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20210518_140306/"
AutoGluon Version:  0.2.0
Train Data Rows:    4088
Train Data Columns: 11
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [1, 0]
	If 'binary' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    11683.03 MB
	Train Data (Original)  Memory Usage: 1.49 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes o

In [50]:
predictor.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                   model  score_val  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0        LightGBM_BAG_L1   0.939335       0.064907   0.975299                0.064907           0.975299            1       True          1
1    WeightedEnsemble_L2   0.939335       0.072867   1.395137                0.007961           0.419838            2       True          3
2  NeuralNetMXNet_BAG_L1   0.939090       1.002739   6.608344                1.002739           6.608344            1       True          2
3        LightGBM_BAG_L2   0.939090       1.140566   8.557316                0.072920           0.973674            2       True          4
4    WeightedEnsemble_L3   0.939090       1.148634   8.987367                0.008068           0.430051            3       True          6
5  NeuralNetMXNet_BAG_L2   0.939090       2.103818  14.326887                1.036172           6.

{'leaderboard':                    model  score_val  ...  can_infer  fit_order
 0        LightGBM_BAG_L1   0.939335  ...       True          1
 1    WeightedEnsemble_L2   0.939335  ...       True          3
 2  NeuralNetMXNet_BAG_L1   0.939090  ...       True          2
 3        LightGBM_BAG_L2   0.939090  ...       True          4
 4    WeightedEnsemble_L3   0.939090  ...       True          6
 5  NeuralNetMXNet_BAG_L2   0.939090  ...       True          5
 
 [6 rows x 9 columns],
 'max_stack_level': 3,
 'model_best': 'WeightedEnsemble_L2',
 'model_fit_times': {'LightGBM_BAG_L1': 0.9752988815307617,
  'LightGBM_BAG_L2': 0.9736735820770264,
  'NeuralNetMXNet_BAG_L1': 6.608343839645386,
  'NeuralNetMXNet_BAG_L2': 6.743243932723999,
  'WeightedEnsemble_L2': 0.41983771324157715,
  'WeightedEnsemble_L3': 0.4300506114959717},
 'model_hyperparams': {'LightGBM_BAG_L1': {'max_base_models': 25,
   'max_base_models_per_type': 5,
   'save_bag_folds': True,
   'use_orig_features': True},
  'Light

In [51]:
predictor.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                   model  score_val  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0        LightGBM_BAG_L1   0.939335       0.064907   0.975299                0.064907           0.975299            1       True          1
1    WeightedEnsemble_L2   0.939335       0.072867   1.395137                0.007961           0.419838            2       True          3
2  NeuralNetMXNet_BAG_L1   0.939090       1.002739   6.608344                1.002739           6.608344            1       True          2
3        LightGBM_BAG_L2   0.939090       1.140566   8.557316                0.072920           0.973674            2       True          4
4    WeightedEnsemble_L3   0.939090       1.148634   8.987367                0.008068           0.430051            3       True          6
5  NeuralNetMXNet_BAG_L2   0.939090       2.103818  14.326887                1.036172           6.

{'leaderboard':                    model  score_val  ...  can_infer  fit_order
 0        LightGBM_BAG_L1   0.939335  ...       True          1
 1    WeightedEnsemble_L2   0.939335  ...       True          3
 2  NeuralNetMXNet_BAG_L1   0.939090  ...       True          2
 3        LightGBM_BAG_L2   0.939090  ...       True          4
 4    WeightedEnsemble_L3   0.939090  ...       True          6
 5  NeuralNetMXNet_BAG_L2   0.939090  ...       True          5
 
 [6 rows x 9 columns],
 'max_stack_level': 3,
 'model_best': 'WeightedEnsemble_L2',
 'model_fit_times': {'LightGBM_BAG_L1': 0.9752988815307617,
  'LightGBM_BAG_L2': 0.9736735820770264,
  'NeuralNetMXNet_BAG_L1': 6.608343839645386,
  'NeuralNetMXNet_BAG_L2': 6.743243932723999,
  'WeightedEnsemble_L2': 0.41983771324157715,
  'WeightedEnsemble_L3': 0.4300506114959717},
 'model_hyperparams': {'LightGBM_BAG_L1': {'max_base_models': 25,
   'max_base_models_per_type': 5,
   'save_bag_folds': True,
   'use_orig_features': True},
  'Light

In [52]:
predictor.leaderboard()

                   model  score_val  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0        LightGBM_BAG_L1   0.939335       0.064907   0.975299                0.064907           0.975299            1       True          1
1    WeightedEnsemble_L2   0.939335       0.072867   1.395137                0.007961           0.419838            2       True          3
2  NeuralNetMXNet_BAG_L1   0.939090       1.002739   6.608344                1.002739           6.608344            1       True          2
3        LightGBM_BAG_L2   0.939090       1.140566   8.557316                0.072920           0.973674            2       True          4
4    WeightedEnsemble_L3   0.939090       1.148634   8.987367                0.008068           0.430051            3       True          6
5  NeuralNetMXNet_BAG_L2   0.939090       2.103818  14.326887                1.036172           6.743244            2       True          5


,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBM_BAG_L1,0.939335,0.064907,0.975299,0.064907,0.975299,1,True,1
1,WeightedEnsemble_L2,0.939335,0.072867,1.395137,0.007961,0.419838,2,True,3
2,NeuralNetMXNet_BAG_L1,0.939090,1.002739,6.608344,1.002739,6.608344,1,True,2
3,LightGBM_BAG_L2,0.939090,1.140566,8.557316,0.072920,0.973674,2,True,4
4,WeightedEnsemble_L3,0.939090,1.148634,8.987367,0.008068,0.430051,3,True,6
5,NeuralNetMXNet_BAG_L2,0.939090,2.103818,14.326887,1.036172,6.743244,2,True,5


In [53]:
predictor.leaderboard(test_data, extra_metrics=['accuracy', 'balanced_accuracy', 'log_loss'], silent=True)

,model,score_test,accuracy,balanced_accuracy,log_loss,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBM_BAG_L1,1.0,1.0,1.0,-0.060895,0.939335,0.071181,0.064907,0.975299,0.071181,0.064907,0.975299,1,True,1
1,WeightedEnsemble_L2,1.0,1.0,1.0,-0.060895,0.939335,0.073492,0.072867,1.395137,0.002311,0.007961,0.419838,2,True,3
2,NeuralNetMXNet_BAG_L1,1.0,1.0,1.0,-0.520756,0.939090,1.142705,1.002739,6.608344,1.142705,1.002739,6.608344,1,True,2
3,LightGBM_BAG_L2,1.0,1.0,1.0,-0.062378,0.939090,1.285447,1.140566,8.557316,0.071561,0.072920,0.973674,2,True,4
4,WeightedEnsemble_L3,1.0,1.0,1.0,-0.062378,0.939090,1.287183,1.148634,8.987367,0.001736,0.008068,0.430051,3,True,6
5,NeuralNetMXNet_BAG_L2,1.0,1.0,1.0,-0.538464,0.939090,2.346700,2.103818,14.326887,1.132814,1.036172,6.743244,2,True,5


In [54]:
predictor.feature_importance(test_data)

Computing feature importance via permutation shuffling for 11 features using 1000 rows with 3 shuffle sets...
	3.87s	= Expected runtime (1.29s per shuffle set)
	0.87s	= Actual runtime (Completed 3 of 3 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
smoking_status,0.0,0.0,0.5,3,0.0,0.0
bmi,0.0,0.0,0.5,3,0.0,0.0
avg_glucose_level,0.0,0.0,0.5,3,0.0,0.0
Residence_type,0.0,0.0,0.5,3,0.0,0.0
work_type,0.0,0.0,0.5,3,0.0,0.0
ever_married,0.0,0.0,0.5,3,0.0,0.0
heart_disease,0.0,0.0,0.5,3,0.0,0.0
hypertension,0.0,0.0,0.5,3,0.0,0.0
age,0.0,0.0,0.5,3,0.0,0.0
gender,0.0,0.0,0.5,3,0.0,0.0
